# STA130 Project Tutorial Assignment 1
**Group member**:
- Vu My Hanh Tran
- Zhaoyang Wang
- Cheryl Tong
- Jiawei Gao

In [4]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd

df_preds = pd.read_csv('test_predictions.csv')
# Prediction Probility "Error"

df_indicators = pd.read_csv('country_indicators.csv')

# using transformer for this example demonstration, as opposed to
# df_preds.y_true_ffnn and df_preds.y_pred_proba_ffnn
# df_preds.y_true_xgboost and df_preds.y_pred_proba_xgboost

# quick and dirty scalars to represent prediction 'wrongness'
# (subtract model's prediction probability from boolean true value)
df_preds['transformer_probability_prediction_error'] = np.abs(df_preds['y_true_transformer'].astype(float) - df_preds['y_pred_proba_transformer'])
df_preds[['y_true_transformer','y_pred_proba_transformer','transformer_probability_prediction_error']]

,y_true_transformer,y_pred_proba_transformer,transformer_probability_prediction_error
0,False,0.183897,0.183897
1,False,0.267831,0.267831
2,False,0.482585,0.482585
3,False,0.187792,0.187792
4,True,0.539319,0.460681
...,...,...,...
359,False,0.182196,0.182196
360,False,0.203236,0.203236
361,False,0.527107,0.527107
362,False,0.555677,0.555677


In [5]:
# Prediction Classification "Correctness"

threshold = 0.5 # using default classification threshold

# using transformer for this example demonstration, as opposed to
# df_preds.y_true_ffnn and df_preds.y_pred_proba_ffnn>threshold
# df_preds.y_true_xgboost and df_preds.y_pred_proba_xgboost>threshold
df_preds['transformer_classifcation_performance_outcome'] = None

tmp = df_preds['transformer_classifcation_performance_outcome'].copy()
TP_pos_pred_correct = df_preds.y_true_transformer & (df_preds.y_pred_proba_transformer>threshold)
tmp[TP_pos_pred_correct] = "correctly predicted escalation"
TN_neg_pred_correct = (~df_preds.y_true_transformer) & (df_preds.y_pred_proba_transformer<=threshold)
tmp[TN_neg_pred_correct] = "correctly predicted no escalation"
FP_pos_pred_wrong = (~df_preds.y_true_transformer) & (df_preds.y_pred_proba_transformer>threshold)
tmp[FP_pos_pred_wrong] = "wrongly predicted escalation"
FN_neg_pred_wrong = df_preds.y_true_transformer & (df_preds.y_pred_proba_transformer<=threshold)
tmp[FN_neg_pred_wrong] = "wrongly predicted no escalation"

df_preds['transformer_classifcation_performance_outcome'] = tmp
df_preds[['y_true_transformer','y_pred_transformer','transformer_classifcation_performance_outcome']][30:50]

,y_true_transformer,y_pred_transformer,transformer_classifcation_performance_outcome
30,True,True,correctly predicted escalation
31,False,True,wrongly predicted escalation
32,False,True,wrongly predicted escalation
33,False,True,wrongly predicted escalation
34,False,False,correctly predicted no escalation
35,False,True,wrongly predicted escalation
36,False,False,correctly predicted no escalation
37,False,False,correctly predicted no escalation
38,False,True,wrongly predicted escalation
39,False,True,wrongly predicted escalation


In [6]:
import numpy as np

# arbitrary simulation parameters
n,R = 5,100

# a first "sample"
sample_1 = np.arange(n)

# bootstrapping a single sample
for rep in range(R):
    bootstrap_sample = np.random.choice(sample_1, size=n)
    # original sample size maintained, of course; and, this function defaults to `replace=True`...

# coin flipping under a specified null hypothesis
for rep in range(R):
    binary_sample = np.random.choice([0,1], p=[0.5,0.5], size=n)

# a second "sample"
sample_2 = np.arange(n,2*n)

# bootstrapping two samples separately
for rep in range(R):
    bootstrap_sample_1 = np.random.choice(sample_1, size=n)
    bootstrap_sample_2 = np.random.choice(sample_2, size=n)
    
# permutation/shuffling of "treatment/group" labels
# corresponds to a null hypothesis that the "treatment/group" labels don't matter
labels = np.array(["sample_1"]*n + ["sample_2"]*n)
all_samples = np.concatenate([sample_1, sample_2], axis=0)
for rep in range(R):
    # `replace=False` is very important
    shuffled_samples = np.random.choice(all_samples, size=n*2, replace=False)
    # keeping the same original sample sizes is very important (as they could be different...)
    shuffled_sample_1 = shuffled_samples[labels=="sample_1"]#[:n]
    shuffled_sample_2 = shuffled_samples[labels=="sample_2"]#[n:]

# "paired" samples analysis
sample_difference = sample_1 - sample_2

# bootstrapping a "paired" sample
for rep in range(R):
    bootstrap_sample = np.random.choice(sample_difference, size=n)

# null hypothesis of "order doesn't matter" for a "paired" sample
for rep in range(R):
    order_of_subtraction = np.random.choice([1,-1], p=[0.5,0.5], size=n)
    sample_difference_random_order = sample_difference*order_of_subtraction

# Question 1

In [7]:
# Calculate prediction probability error for xgboost
df_preds['xgboost_probability_prediction_error'] = np.abs(df_preds['y_true_xgboost'].astype(float) - df_preds['y_pred_proba_xgboost'])

# Calculate prediction probability error for ffnn
df_preds['ffnn_probability_prediction_error'] = np.abs(df_preds['y_true_ffnn'].astype(float) - df_preds['y_pred_proba_ffnn'])

In [8]:
df_preds['xgboost_probability_prediction_error'] 

0      0.066500
1      0.099643
2      0.295914
3      0.361556
4      0.608380
         ...   
359    0.079453
360    0.060189
361    0.302375
362    0.729246
363    0.591722
Name: xgboost_probability_prediction_error, Length: 364, dtype: float64

In [9]:
df_preds['ffnn_probability_prediction_error'] 

0      0.409958
1      0.406696
2      0.545236
3      0.534560
4      0.461417
         ...   
359    0.291874
360    0.300321
361    0.335496
362    0.324000
363    0.667545
Name: ffnn_probability_prediction_error, Length: 364, dtype: float64

# Question 2

In [10]:
model_name = 'xgboost'
np.random.seed(100)

# Extract the prediction probability error values for the chosen model
prediction_error = df_preds[f'{model_name}_probability_prediction_error'].values

# Set the number of bootstrap samples
n_bootstrap = 1000

# Initialize an array to store the bootstrap sample means
bootstrap_sample_means = np.zeros(n_bootstrap)

# Perform bootstrap resampling
for i in range(n_bootstrap):
    # Generate a bootstrap sample by sampling with replacement
    bootstrap_sample = np.random.choice(prediction_error, size=len(prediction_error), replace=True)
    
    # Calculate the mean of the bootstrap sample
    bootstrap_sample_means[i] = np.mean(bootstrap_sample)

# Calculate the 95% confidence interval of the bootstrap sample means
calc_interval = tuple(np.quantile(bootstrap_sample_means,(.025, .975)))
calc_interval

(0.4224395601175972, 0.468324271573825)

# Question 3

In [11]:
# Create Prediction Classification "Correctness" results for transformer
df_preds['transformer_correctness'] = np.where(df_preds['y_true_transformer'] == df_preds['y_pred_transformer'], 'correct', 'incorrect')

# Create Prediction Classification "Correctness" results for xgboost
df_preds['xgboost_correctness'] = np.where(df_preds['y_true_xgboost'] == df_preds['y_pred_xgboost'], 'correct', 'incorrect')

# Create Prediction Classification "Correctness" results for ffnn
df_preds['ffnn_correctness'] = np.where(df_preds['y_true_ffnn'] == df_preds['y_pred_ffnn'], 'correct', 'incorrect')

In [12]:
df_preds['transformer_correctness']

0        correct
1        correct
2        correct
3        correct
4        correct
         ...    
359      correct
360      correct
361    incorrect
362    incorrect
363      correct
Name: transformer_correctness, Length: 364, dtype: object

In [13]:
df_preds['xgboost_correctness']

0        correct
1        correct
2        correct
3        correct
4      incorrect
         ...    
359      correct
360      correct
361      correct
362    incorrect
363    incorrect
Name: xgboost_correctness, Length: 364, dtype: object

In [14]:
df_preds['ffnn_correctness']

0        correct
1        correct
2      incorrect
3      incorrect
4        correct
         ...    
359      correct
360      correct
361      correct
362      correct
363    incorrect
Name: ffnn_correctness, Length: 364, dtype: object

# Question 4

In [15]:
from scipy.stats import binomtest

# Select the model for which to perform the hypothesis test
selected_model = 'xgboost'

# Determine the null hypothesis
null_proportion = 0.4

# Extract the Prediction Classification "Correctness" results for the selected model
correctness = df_preds[f'{selected_model}_correctness']

# Calculate the proportion of the specific Prediction Classification "Correctness" category
proportion = correctness.value_counts(normalize=True)['incorrect']

# Perform the hypothesis test using the binomtest function
p_value = binomtest(int(proportion * len(correctness)), n=len(correctness), p=null_proportion).pvalue

In [16]:
p_value

0.0002681471561002832

> The calculated p-value for the one-sample hypothesis test is approximately 0.0003. Based on this result, we have very strong evidence to reject the null hypothesis.

# Question 5

In [17]:
# This subset consists of "high income" countries, is created with reference to the wbi_income_group category
df = df_preds.merge(df_indicators, left_on='iso3', right_on='iso3', how='inner')
high_income = df[df.wbi_income_group == 'High income']
high_income

,yearmonth,fips,y_pred_transformer,y_pred_proba_transformer,y_true_transformer,y_pred_xgboost,y_pred_proba_xgboost,y_true_xgboost,y_pred_ffnn,y_pred_proba_ffnn,...,fsi_e3:_human_flight_and_brain_drain,fsi_e2:_economic_inequality,fsi_e1:_economy,fsi_p1:_state_legitimacy,fsi_p2:_public_services,fsi_p3:_human_rights,fsi_c1:_security_apparatus,fsi_c2:_factionalized_elites,fsi_x1:_external_intervention,fsi_category
5,202211,CA,False,0.360460,False,False,0.328418,False,False,0.298536,...,0.7,2.5,1.4,0.4,1.7,1.9,2.2,2.5,0.4,Sustainable
6,202212,CA,False,0.421430,False,False,0.294212,False,False,0.299269,...,0.7,2.5,1.4,0.4,1.7,1.9,2.2,2.5,0.4,Sustainable
7,202211,US,True,0.701943,False,False,0.436748,False,False,0.325844,...,1.7,4.0,2.1,4.2,2.2,4.2,4.7,7.3,1.0,Stable
8,202212,US,True,0.715726,False,False,0.438737,True,False,0.326096,...,1.7,4.0,2.1,4.2,2.2,4.2,4.7,7.3,1.0,Stable
9,202301,US,True,0.749787,False,False,0.405529,False,False,0.325571,...,1.7,4.0,2.1,4.2,2.2,4.2,4.7,7.3,1.0,Stable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,202212,CY,False,0.199646,False,False,0.107952,False,False,0.282298,...,3.5,4.5,5.6,4.2,3.3,3.0,2.9,7.9,8.3,Stable
336,202301,CY,False,0.182541,False,False,0.312885,False,False,0.288410,...,3.5,4.5,5.6,4.2,3.3,3.0,2.9,7.9,8.3,Stable
361,202211,TD,True,0.527107,False,True,0.697625,True,False,0.335496,...,6.8,4.7,3.9,3.5,4.3,3.5,7.3,5.6,3.3,Stable
362,202212,TD,True,0.555677,False,True,0.729246,False,False,0.324000,...,6.8,4.7,3.9,3.5,4.3,3.5,7.3,5.6,3.3,Stable


In [18]:
# There are 100 countries with a "high income" remark in the wbi_income_group category
df.wbi_income_group.value_counts()

Upper middle income    103
Lower middle income    102
High income            100
Low income              58
Name: wbi_income_group, dtype: int64

In [19]:
model_name = 'xgboost'
np.random.seed(100)

# Extract the prediction probability error values for the chosen model
prediction_error_high_income = high_income[f'{model_name}_probability_prediction_error'].values

# Set the number of bootstrap samples
n_bootstrap = 1000

# Initialize an array to store the bootstrap sample means
bootstrap_sample_means_high_income = np.zeros(n_bootstrap)

# Perform bootstrap resampling
for i in range(n_bootstrap):
    # Generate a bootstrap sample by sampling with replacement
    bootstrap_sample_high_income = np.random.choice(prediction_error_high_income, size=len(prediction_error_high_income), replace=True)
    
    # Calculate the mean of the bootstrap sample
    bootstrap_sample_means_high_income[i] = np.mean(bootstrap_sample_high_income)

# Calculate the 95% confidence interval of the bootstrap sample means
calc_interval_high_income = tuple(np.round(np.quantile(bootstrap_sample_means_high_income,(.025, .975)), 3))
calc_interval_high_income

(0.236, 0.32)

> For countries lying on the "high income" group in terms of wbi_income_group, there is a 95% chance for the mean of probability prediction error to lie between 0.236 and 0.32.

# Question 6

In [20]:
# This subset consists of "SSA" countries, is created with reference to the hdr_region category
SSA = df[df.hdr_region == 'SSA']
SSA

,yearmonth,fips,y_pred_transformer,y_pred_proba_transformer,y_true_transformer,y_pred_xgboost,y_pred_proba_xgboost,y_true_xgboost,y_pred_ffnn,y_pred_proba_ffnn,...,fsi_e3:_human_flight_and_brain_drain,fsi_e2:_economic_inequality,fsi_e1:_economy,fsi_p1:_state_legitimacy,fsi_p2:_public_services,fsi_p3:_human_rights,fsi_c1:_security_apparatus,fsi_c2:_factionalized_elites,fsi_x1:_external_intervention,fsi_category
28,202211,KE,True,0.780935,False,True,0.809601,True,True,0.512650,...,6.4,7.4,6.4,7.3,8.0,6.3,6.8,8.9,6.9,Warning
29,202212,KE,True,0.626850,False,True,0.666303,True,True,0.511990,...,6.4,7.4,6.4,7.3,8.0,6.3,6.8,8.9,6.9,Warning
30,202301,KE,True,0.692808,True,True,0.804359,False,True,0.507454,...,6.4,7.4,6.4,7.3,8.0,6.3,6.8,8.9,6.9,Warning
34,202211,CD,False,0.182014,False,False,0.402866,False,True,0.630258,...,7.7,8.7,8.4,9.1,9.6,8.4,8.7,9.5,7.9,Alert
35,202212,CD,True,0.559438,False,True,0.544280,False,True,0.630537,...,7.7,8.7,8.4,9.1,9.6,8.4,8.7,9.5,7.9,Alert
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,202212,UG,True,0.549558,True,True,0.807170,False,True,0.566986,...,6.1,7.4,6.4,8.2,8.0,7.2,6.3,8.9,7.3,Alert
349,202301,UG,True,0.577561,False,True,0.683059,False,True,0.565887,...,6.1,7.4,6.4,8.2,8.0,7.2,6.3,8.9,7.3,Alert
350,202211,RW,True,0.575970,False,True,0.686765,True,True,0.511833,...,6.1,7.8,6.4,6.7,6.7,6.2,5.2,8.0,5.7,Warning
351,202212,RW,True,0.562719,True,True,0.653654,False,True,0.525747,...,6.1,7.8,6.4,6.7,6.7,6.2,5.2,8.0,5.7,Warning


In [21]:
# There are 87 countries with a "SSA" remark in the hdr_region category
df.hdr_region.value_counts()

SSA    87
LAC    63
AS     39
ECA    38
EAP    27
SA     20
Name: hdr_region, dtype: int64

In [22]:
model_name = 'xgboost'
np.random.seed(100)

# Extract the prediction probability error values for the chosen model
prediction_error_SSA = SSA[f'{model_name}_probability_prediction_error'].values

# Set the number of bootstrap samples
n_bootstrap = 1000

# Initialize an array to store the bootstrap sample means
bootstrap_sample_means_SSA = np.zeros(n_bootstrap)

# Perform bootstrap resampling
for i in range(n_bootstrap):
    # Generate a bootstrap sample by sampling with replacement
    bootstrap_sample_SSA = np.random.choice(prediction_error_SSA, size=len(prediction_error_SSA), replace=True)
    
    # Calculate the mean of the bootstrap sample
    bootstrap_sample_means_SSA[i] = np.mean(bootstrap_sample_SSA)

# Calculate the 95% confidence interval of the bootstrap sample means
calc_interval_SSA = tuple(np.round(np.quantile(bootstrap_sample_means_SSA,(.025, .975)), 3))
calc_interval_SSA

(0.51, 0.581)

In [23]:
bootstrap_sample_means_difference = bootstrap_sample_means_SSA - bootstrap_sample_means_high_income
bootstrap_sample_means_difference

calc_interval_difference = tuple(np.round(np.quantile(bootstrap_sample_means_difference,(.025, .975)), 3))
calc_interval_difference

(0.218, 0.319)

> When comparing the bootstrapped means of the probability prediction error for countries according to their wbi_income_group and hdr_region, there is a 95% chance for the difference to lie in between 0.218 and 0.319.

# Question 7

- Null Hypothesis: The probability prediction error means in different subsets are equal to each other.
- Alternative Hypothesis: The probability prediction error means in different subsets aren't equal to each other.

In [24]:
from scipy import stats
# I used mannwhitneyu test to find out the p-value
p_value = stats.mannwhitneyu(df.xgboost_probability_prediction_error[df.wbi_income_group == 'High income'], 
                             df.xgboost_probability_prediction_error[df.hdr_region == 'SSA'])[1]
p_value

6.08105179915781e-15

> When referring the p-value, which is the proportion of permuted test statistics that are as extreme as or more extreme than the actual test statistic, to the significance level, we can conclude that we have an extremely strong evidence against the null hypothesis.

# Question 8


In [29]:
sample_transformer=df['transformer_probability_prediction_error'].to_numpy()
sample_xgboost=df['xgboost_probability_prediction_error'].to_numpy()
sample_difference = sample_transformer - sample_xgboost

n,R = len(sample_1), 1000
difference_test_stats = np.zeros(R)
for rep in range(R):
    bootstrap_sample = np.random.choice(sample_difference, size=n)
    difference_test_stats[rep] = np.mean(bootstrap_sample)
    confidence_interval = np.percentile(difference_test_stats, [2.5, 97.5])

confidence_interval

array([-0.026992  ,  0.01704544])

In [34]:
import numpy as np
sample_1 = df_preds['ffnn_probability_prediction_error'].to_numpy()
sample_2 = df_preds['xgboost_probability_prediction_error'].to_numpy ()
sample_difference = sample_1 - sample_2
R = 1000
n = len(sample_difference)
observed_mean_diff = np. mean(sample_difference)
observed_mean_diff

-0.028889081111447595

# Question 9

In [32]:
import numpy as np
sample_transformer = df_preds['transformer_probability_prediction_error'].to_numpy()
sample_xgboost = df_preds['xgboost_probability_prediction_error'].to_numpy()
sample_difference = sample_transformer - sample_xgboost

R = 1000
n = len(sample_difference)
observed_mean_diff = np.mean(sample_difference)
mean_diffs = np.empty(R)

for rep in range(R):
    order_of_subtraction = np.random.choice([1, -1], size=n)
    sample_difference_random = sample_difference * order_of_subtraction
    mean_diffs[rep] = np.mean(sample_difference_random_order)
    p_value = np.mean(abs(mean_diffs) >= abs(observed_mean_diff))

p_value

1.0